<a href="https://colab.research.google.com/github/nshah-waripari/nlp_transformers/blob/main/nmt_nepali_english_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## English to Nepali Translation with T5 Transformer Model

In [1]:
# Install the required libraries
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter2=True)

Cloning into 'notebooks'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (422/422), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 422 (delta 197), reused 411 (delta 191), pack-reused 0
Receiving objects: 100% (422/422), 24.99 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (197/197), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


### Load the dataset
1. We will be using data [Nepali - English Language Pair] obtained from The Tatoeba Translation Challenge [https://github.com/Helsinki-NLP/Tatoeba-Challenge].
2. Train SentencePiece model [https://github.com/google/sentencepiece] to tokenize the raw data and use Tokenizer[from HuggingFace] to tokenize the text that will be language agnostic.


In [9]:
# first mount the drive
from google.colab import drive
drive.mount('content')

# load the training & test dataset, pre-trained spm model along with vocab file hosted at dropbox
training_dataset_url = "https://www.dropbox.com/s/oey08zbsdsqjhqb/train.tsv.zip?dl=0" # train.tsv.zip this is a zip file, need to unzip before using it
eval_dataset_url = "https://www.dropbox.com/s/wbxha2sm4hitn6a/eval.tsv?dl=0" # eval.tsv file
spm_model = "https://www.dropbox.com/s/divi5qh1atzpu5p/spm.model?dl=0" # spm.model
vocab = "https://www.dropbox.com/s/k129w9ylkosrxnv/spm.vocab?dl=0" #spm.vocab
# download training.tsv.zip file
!wget -P data/ {training_dataset_url}
# download eval.tsv file
!wget -P data/ {eval_dataset_url}
# download spm.model file
!wget -P data/ {spm_model}
# download vocab file
!wget -P data/ {vocab}

Drive already mounted at content; to attempt to forcibly remount, call drive.mount("content", force_remount=True).
--2022-08-06 15:09:04--  https://www.dropbox.com/s/oey08zbsdsqjhqb/train.tsv.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/oey08zbsdsqjhqb/train.tsv.zip [following]
--2022-08-06 15:09:05--  https://www.dropbox.com/s/raw/oey08zbsdsqjhqb/train.tsv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9b659be41decfb59a516ef85eb.dl.dropboxusercontent.com/cd/0/inline/BqgZ2mk9zAOVh1I4ZqKNF_tBEVNsDsK20GYiKkjW-Fv-PNhT3rXVVoMAFbXf4M2DMKL6_D_R1uQem8UicfrCH21afMaRAz0q4FeAZ3FCnNuoB3rzbV2n2W9WbkryS1IdTXrX6x-zb806AEN38r5LaTq9cTO3EmEYgFlTS712as3esQ/file# [following]
--2022-08-06 15:09:05--  https://uc9b659be

### Prepare Dataset

In [25]:
# !ls data
# !unzip data/train.tsv.zip?dl=0 -d "data"
!head -n 5 train.tsv



	prefix	input_text	target_text
0	translate nepali to english	'सिहर्स-अभिकर्ता' कार्यक्रम असफलतापूर्ण अन्त्य भयो ।	The 'seahorse-agent' program exited unsuccessfully.
1	translate english to nepali	The 'seahorse-agent' program exited unsuccessfully.	'सिहर्स-अभिकर्ता' कार्यक्रम असफलतापूर्ण अन्त्य भयो ।
2	translate nepali to english	अन्य कोठा	Other Rooms
3	translate english to nepali	Other Rooms	अन्य कोठा


In [28]:
from datasets import load_dataset
nep_eng_ds = load_dataset("csv", data_files="data/train.tsv", sep="\t", names=["prefix", "input_text", "target_text"])


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9597c4c534fbc676/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
nep_eng_ds["train"][0]

{'__index_level_0__': None,
 'input_text': 'input_text',
 'prefix': 'prefix',
 'target_text': 'target_text'}

In [36]:
print(nep_eng_ds.column_names)

{'train': ['prefix', 'input_text', 'target_text', '__index_level_0__']}


From Datasets to DataFrame

In [ ]:
import pandas as pd
train_ds.set_format(type="pandas")
df = train_ds[:]
df.head()


,id,translation
0,0,"{'en': 'Add Feed to Akregator', 'ne': 'एक्रिगे..."
1,1,"{'en': 'Add Feeds to Akregator', 'ne': 'एक्रिग..."
2,2,"{'en': 'Add All Found Feeds to Akregator', 'ne..."
3,3,{'en': 'Subscribe to site updates (using news ...
4,4,"{'en': 'Imported Feeds', 'ne': 'आयातित फिड'}"


In [ ]:
from transformers import pipeline
model_checkpoint = "Helsinki-NLP/opus-mt-en-ne"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

RepositoryNotFoundError: ignored

In [ ]:
from transformers import T5ForConditionalGeneration

ImportError: ignored

In [ ]:
import sentencepiece as spm

ModuleNotFoundError: ignored